In [1]:
import torch
print(torch.__version__)       # 应该是 2.7.x / 2.8.x，不带 +cpu
print(torch.version.cuda)      # 应该显示 12.1
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # 你的 5070Ti


2.10.0.dev20251002+cu128
12.8
True
NVIDIA GeForce RTX 5070 Ti Laptop GPU


In [2]:
import pandas as pd

df = pd.read_csv("../../data/lr_results/all_slides_lr_pairs.csv")

# 只取需要的列
df = df[["partner_a", "partner_b"]]

# 规范化顺序：保证 partner_a 始终是字母序小的
df = pd.DataFrame(
    df.apply(lambda row: sorted([row["partner_a"], row["partner_b"]]), axis=1).tolist(),
    columns=["partner_a", "partner_b"]
)

# 去重
df = df.drop_duplicates()

# 保存
df.to_csv("../../data/lr_results/clean_lr_pairs.csv", index=False)
print("处理完成，结果保存在 1_clean.csv")


处理完成，结果保存在 1_clean.csv


In [2]:
import numpy as np

# 语料
text = "面壁者 破壁人 三体 人类 宇宙".split()
vocab = list(set(text))
stoi = {w:i for i,w in enumerate(vocab)}
ids = np.array([stoi[w] for w in text])
V, D = len(vocab), 8

# 初始化词向量
W_in  = np.random.randn(V, D) * 0.1
W_out = np.random.randn(V, D) * 0.1

def sigmoid(x): return 1/(1+np.exp(-x))

# Skip-gram + 负采样
for epoch in range(200):
    loss = 0
    for i, c in enumerate(ids):
        ctx = [ids[j] for j in [i-1,i+1] if 0<=j<len(ids)]
        for t in ctx:
            neg = np.random.randint(0,V,3)  # 负样本3个
            for target,label in [(t,1)] + [(n,0) for n in neg]:
                dot = W_in[c] @ W_out[target]
                p = sigmoid(dot)
                grad = (label - p)
                W_in[c]  += 0.05 * grad * W_out[target]
                W_out[target] += 0.05 * grad * W_in[c]
                loss += -np.log(p if label else (1-p)+1e-9)
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, loss={loss:.3f}")

# 语义相似度展示
def most_similar(word):
    i = stoi[word]
    sim = W_in @ W_in[i]
    j = np.argsort(-sim)
    print(f"'{word}' 的相似词:", [vocab[k] for k in j[:3]])

most_similar("人类")


Epoch 0, loss=22.355
Epoch 50, loss=16.114
Epoch 100, loss=17.700
Epoch 150, loss=13.824
'人类' 的相似词: ['人类', '破壁人', '面壁者']
